# Question 2

Superior Roasters Coffee Company mixes specialty blends of coffee to sell to Big Cup Coffee, a small chain of coffee shops. The beans used in their specialty coffee are listed in the table below:

| Bean Type       | Cost / Pound ($)           | Available amount (lbs)  |
| ------------- |:-------------:| :-----:|
| Columbian      | 1.00 | 550 |
| Brazilian      | 0.85     |   450 |
| Sumatran | 1.55    |    650 |

Superior Roaster's products are:

- Robust Joe must consist of 60-75% Sumatran beans and at least 10% Columbian beans. Each pound of Robust Joe can be sold to Big Cup for $4.25.

- Light Joe must consist of 50-60% Brazilian beans and no more than 20% Sumatran beans. Each pound of Light Joe can be sold to Big Cup for $3.95.

Formulate an LP to maximize the total profit (revenue - cost). 

In [1]:
# symbol vectors for types of beans and the two blends 
types = [:Columbian, :Brazilian, :Sumatran];
blends = [:Robust, :Light];

In [2]:
using JuMP
using HiGHS

# create model with appropriate optimizer 
coffeeblend = Model(HiGHS.Optimizer);

In [5]:
# cost per pound of each kind of bean (in $)
coffee_price = Dict(:Columbian => 1.00, :Brazilian => 0.85, :Sumatran => 1.55)

# amount of each kind of bean available (in lbs)
bean_availability = Dict(:Columbian => 550, :Brazilian => 450, :Sumatran => 650)

# money made from each type of blend sold (in $)
purchase_price = Dict(:Robust => 4.25, :Light => 3.95)

#Quantity available of each resource
# num_labor = 4000
# num_metal = 5000

bean_availability

Dict{Symbol, Int64} with 3 entries:
  :Sumatran  => 650
  :Columbian => 550
  :Brazilian => 450

In [6]:
# declaring variable that is indexed over types and blends
x = @variable(coffeeblend, blends[types] >= 0);

x

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [:Columbian, :Brazilian, :Sumatran]
And data, a 3-element Vector{VariableRef}:
 blends[Columbian]
 blends[Brazilian]
 blends[Sumatran]

In [ ]:
# constraints for Robust Joe
@constraint(coffeeblend, RSLowerBound, x[:Sumatran, :Robust] >= 0.6*sum(x[i,:Robust] for i in types));
@constraint(coffeeblend, RSUpperBound, x[:Sumatran, :Robust] <= 0.75*sum(x[i,:Robust] for i in types));
@constraint(coffeeblend, RCBound, x[:Columbian, :Robust] >= 0.1*sum(x[i,:Robust] for i in types));

# constraints for Light Joe
@constraint(coffeeblend, LBLowerBound, x[:Brazilian, :Light] >= 0.5*sum(x[i,:Light] for i in types));
@constraint(coffeeblend, LBUpperBound, x[:Brazilian, :Light] <= 0.6*sum(x[i,:Light] for i in types));
@constraint(coffeeblend, LSBound, x[:Sumatran, :Robust] <= 0.2*sum(x[i,:Light] for i in types));

In [ ]:
#Create our objective function and set it for minimization
@objective(coffeeblend, Max, sum(profit[i]*frame[i] for i in types));

print(picframe2)

In [ ]:
#Solve the model
optimize!(picframe3);
#Outputs detailed information about the solution process
@show solution_summary(picframe3);